# Traffic fines dataset preparation

In [ ]:
# Skip this cell, at first
import os
os.chdir('../..')
os.getcwd()

In [1]:
import pandas as pd
import numpy as np
from CI_Experiments.config import PROJECT_DIR # if this line results ModuleNotFoundError, then execute the cell above

In [2]:
UNPREPARED_PATH = PROJECT_DIR / 'data/unprepared_process_logs'
PRODUCTION_DATA = UNPREPARED_PATH / 'traffic_fines.csv'
DESTINATION_DATA = PROJECT_DIR / 'data/prepared_process_logs/traffic_fines/activity/activity.csv'

In [3]:
unprepared_data = pd.read_csv(PRODUCTION_DATA, sep=';')
unprepared_data.head()

/var/folders/2v/0cy8xvx12vl1y6x611k263_w0000gn/T/ipykernel_84821/2385976885.py:1: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  unprepared_data = pd.read_csv(PRODUCTION_DATA, sep=';')


,article,vehicleClass,amount,points,Case ID,Activity,Resource,lastSent,notificationType,dismissal,...,Complete Timestamp,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,open_cases,label
0,157.0,A,35.0,0.0,A100,Create Fine,561.0,missing,missing,NIL,...,2006-08-02 01:00:00,60,8,2,1,0.0,0.0,1,11246,deviant
1,157.0,A,35.0,0.0,A100,Send Fine,561.0,missing,missing,NIL,...,2006-12-12 01:00:00,60,12,1,1,190080.0,190080.0,2,12380,deviant
2,157.0,A,35.0,0.0,A100,Insert Fine Notification,561.0,P,P,NIL,...,2007-01-15 01:00:00,60,1,0,1,48960.0,239040.0,3,12202,deviant
3,157.0,A,71.5,0.0,A100,Add penalty,561.0,P,P,NIL,...,2007-03-16 01:00:00,60,3,4,1,86400.0,325440.0,4,7749,deviant
4,157.0,A,36.0,0.0,A10000,Create Fine,561.0,missing,missing,NIL,...,2007-03-09 01:00:00,60,3,4,1,0.0,0.0,1,7756,regular


In [4]:
unprepared_data.columns

Index(['article', 'vehicleClass', 'amount', 'points', 'Case ID', 'Activity',
       'Resource', 'lastSent', 'notificationType', 'dismissal', 'expense',
       'Complete Timestamp', 'timesincemidnight', 'month', 'weekday', 'hour',
       'timesincelastevent', 'timesincecasestart', 'event_nr', 'open_cases',
       'label'],
      dtype='object')

In [5]:
four_columns_for_activity = [
    'Case ID',
    'label',
    'Activity',
    'Complete Timestamp'
]

In [6]:
activity_data = unprepared_data[four_columns_for_activity]
activity_data.head()

,Case ID,label,Activity,Complete Timestamp
0,A100,deviant,Create Fine,2006-08-02 01:00:00
1,A100,deviant,Send Fine,2006-12-12 01:00:00
2,A100,deviant,Insert Fine Notification,2007-01-15 01:00:00
3,A100,deviant,Add penalty,2007-03-16 01:00:00
4,A10000,regular,Create Fine,2007-03-09 01:00:00


In [7]:
activity_data['label'] = np.where(activity_data['label'] == 'deviant', 1, 0)
activity_data.head()

/var/folders/2v/0cy8xvx12vl1y6x611k263_w0000gn/T/ipykernel_84821/2959549615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_data['label'] = np.where(activity_data['label'] == 'deviant', 1, 0)


,Case ID,label,Activity,Complete Timestamp
0,A100,1,Create Fine,2006-08-02 01:00:00
1,A100,1,Send Fine,2006-12-12 01:00:00
2,A100,1,Insert Fine Notification,2007-01-15 01:00:00
3,A100,1,Add penalty,2007-03-16 01:00:00
4,A10000,0,Create Fine,2007-03-09 01:00:00


In [8]:
activity_data = activity_data.rename(columns={'label': 'Outcome', 'Complete Timestamp': 'Timestamp'})
activity_data.head()

,Case ID,Outcome,Activity,Timestamp
0,A100,1,Create Fine,2006-08-02 01:00:00
1,A100,1,Send Fine,2006-12-12 01:00:00
2,A100,1,Insert Fine Notification,2007-01-15 01:00:00
3,A100,1,Add penalty,2007-03-16 01:00:00
4,A10000,0,Create Fine,2007-03-09 01:00:00


In [9]:
len(activity_data['Activity'].unique())

10

In [10]:
np.mean(activity_data.groupby('Case ID').count()['Activity'])

3.5532615823785827

In [13]:
activity_data.to_csv(DESTINATION_DATA, index=False)